In [22]:
%load_ext autoreload
%autoreload 2

# DATA MANIPULATION
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# DATA VISUALISATION
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

#Tenserflow
import tensorflow.keras as tk

from tensorflow.keras import Sequential, layers, regularizers

from tensorflow.keras.utils import pad_sequences

from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten

from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer

# VIEWING OPTIONS IN THE NOTEBOOK
from sklearn import set_config; set_config(display='diagram')


from sklearn.model_selection import train_test_split, KFold

from sklearn.preprocessing import StandardScaler



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert 🔠 words to 🔢 vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>

_Embeddings_ are representations of words using vectors. These embeddings can be learned within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representations into a Recurrent Neural Network. 

▶️ Run this cell and make sure the version of 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) you are using is ≥ 4.0!

In [2]:
!pip freeze | grep gensim

gensim==4.2.0


In [3]:
!pip freeze | grep tensorflow

tensorflow==2.10.0
tensorflow-datasets==4.6.0
tensorflow-estimator==2.10.0
tensorflow-io-gcs-filesystem==0.27.0
tensorflow-metadata==1.10.0


# The data

Keras provides many datasets, among which is the IMDB dataset 🎬:
- It is comprised of sentences that are ***movie reviews***. 
- Each of these reviews is related to a score given by the reviewer.

❓ **Question** ❓ First of all, let's load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer can handle it. Otherwise, rerun with a lower number.  

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [4]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=25)

2023-03-07 14:01:24.525122: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-07 14:01:24.525305: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 14:01:24.525389: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-07 14:01:24.525445: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-07 14:01:24.525493: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

<b><u>Embeddings in the previous challenge</u></b>:

In the previous exercise, we jointly learned a representation for the words, and fed this representation to a RNN, as shown down below 👇: 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which slows down and increases the difficulty of convergence!

<b><u>Embeddings in the current challenge</u></b>:

For this reason, we will separate the steps of learning the word representation and feeding it into a RNN. As shown here: 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with Word2Vec.

The drawback is indeed that the learned embeddings are not _specifically_ designed for our task. However, learning them independently of the task at hand (sentiment analysis) has some advantages: 
- it is very fast to do in general (with Word2Vec)
- the representation learned by Word2Vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with Word2Vec and see how meaningful it is!

# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 🐶 _dog_ $\rightarrow$ [0.1, -0.3, 0.8]
- 🐱 _cat_ $\rightarrow$ [-1.1, 2.3, 0.7]
- 🍏 _apple_ $\rightarrow$ [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

***What is a "good" numerical representation of words?***

- ***Words with close meanings should be geometrically close in your embedding space!***

    - Look at the following example which represents a bi-dimensional embedding space.

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! 

[📚 **Gensim**](https://radimrehurek.com/gensim/)  is a great Python package that makes the use of the Word2Vec algorithm easy to implement, fast and accurate (which is not an easy task!).

1. The following code imports Word2Vec from Gensim. 

2. The second line learns the embedding representation of the words thanks to the sentences in `X_train`. 
3. The third line stores the words and their trained embeddings in `wv`.

In [5]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)
wv = word2vec.wv

Let's look at the embedded representation of some words.

You can use `wv` as a dictionary.
For instance, `wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation was not learned). 

In [6]:
wv['megatron']

KeyError: "Key 'megatron' not present"

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [7]:
len(wv)

14416

🧐 How do we know whether this embedding make any sense or not? 

💡 To investigate this question, we will check that words with a close meaning have close representations. 

👉 Let's use the [**`Word2Vec.wv.most_similar`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar) method that, given an input word, displays the "closest" words in the embedding space. If the embedding is well done, then words with similar meanings will have similar representation in the embedding space.

❓ **Question** ❓ Try out the `most_similar` method on different words. 

🧑🏿‍🏫 The quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [26]:
wv.most_similar('dragon')



[('bases', 0.9206402897834778),
 ('burnt', 0.9192240238189697),
 ('rvd', 0.917300820350647),
 ('swing', 0.9172818064689636),
 ('limbs', 0.9162314534187317),
 ('dudley', 0.9146203398704529),
 ('primal', 0.9140545129776001),
 ('boarding', 0.913859486579895),
 ('industrial', 0.9135359525680542),
 ('forest', 0.9130655527114868)]

📚 Similarly to `most_similar` used on words directly, we can use [**`similar_by_vector`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similar_by_vector) on vectors to do the same thing:

In [27]:
wv.similar_by_vector('dragon')

[('bases', 0.9206402897834778),
 ('burnt', 0.9192240238189697),
 ('rvd', 0.917300820350647),
 ('swing', 0.9172818064689636),
 ('limbs', 0.9162314534187317),
 ('dudley', 0.9146203398704529),
 ('primal', 0.9140545129776001),
 ('boarding', 0.913859486579895),
 ('industrial', 0.9135359525680542),
 ('forest', 0.9130655527114868)]

# Arithmetic on words

Now, let's perform some mathematical operations on words, i.e. on their vector representations!

As any learned word is represented as a vector, you can do basic arithmetic operations, such as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [10]:
wv['good']-wv['bad']

array([ 0.13821852,  0.29293764,  0.31630915, -0.02461898, -0.59089386,
        0.10915166, -0.59043   , -0.4498074 ,  0.08736414,  0.25280398,
        0.07337239,  1.4182571 ,  1.5282569 , -0.1287528 ,  0.4085245 ,
        1.246073  , -0.59631455,  0.4918037 , -0.5533391 , -0.10912788,
       -1.0415547 , -0.10327069,  0.23228982, -1.5964236 , -1.0605935 ,
       -0.6932473 ,  0.09098893,  1.3175317 , -1.6252261 , -0.31663683,
       -0.07955736, -0.35801625, -0.2625772 ,  0.803452  , -0.36748305,
       -0.85303843,  0.2943256 ,  0.28989363,  0.13500118,  1.7001845 ,
       -0.5963145 , -1.4560372 , -1.1599425 ,  0.01694405,  0.2249378 ,
        0.11155123,  0.91058713, -0.39211848, -0.88547236,  1.1530259 ,
        0.44481432, -0.9026528 , -0.35015047, -0.8633362 ,  1.1442974 ,
        0.1154869 , -0.14281133,  0.24031207, -0.11758047, -0.6391529 ,
        0.31546512, -1.1679293 ,  0.5442826 ,  0.30026585,  1.2727592 ,
       -0.14412999, -0.28882372,  1.2253339 ,  0.15873563,  0.32

Now, imagine for a second that the following equality holds true:

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to:

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be bold of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [11]:
res = wv['good']-wv['bad']+wv['stupid']

We said earlier, that for any vector it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vectors of `res`

💡 _Hint_: `similar_by_vector`

In [12]:
wv.similar_by_vector(res)

[('nice', 0.7363389730453491),
 ('good', 0.7231959700584412),
 ('decent', 0.6979979276657104),
 ('great', 0.6316981911659241),
 ('fine', 0.6133387684822083),
 ('interesting', 0.6066603064537048),
 ('strange', 0.5992788076400757),
 ('cool', 0.5874645709991455),
 ('funny', 0.5812341570854187),
 ('beautiful', 0.5655096769332886)]

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [13]:
wv.similar_by_vector(wv['boy']-wv['girl']+wv['woman'])

[('woman', 0.8960340619087219),
 ('boy', 0.8583593964576721),
 ('lady', 0.8326420783996582),
 ('named', 0.8040511012077332),
 ('doctor', 0.8011386394500732),
 ('man', 0.8005926012992859),
 ('soldier', 0.7975231409072876),
 ('rich', 0.7902207970619202),
 ('scientist', 0.7834914326667786),
 ('ex', 0.7781528830528259)]

<u><i>Some notes about Word2Vec as an internal Neural Network</i></u>:

You might wonder where does this magic comes from (at quite a low price, you just ran a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)`, actually trains a internal neural network (that you don't see).  

In a nutshell, this internal neural network predicts a word from the surroundings words in a sentences. Hence, it splits the original sentences, then for each split it chooses some words as inputs $X$ and a word as the output $y$ which it tries to predict, using the embedding space.

And as with any neural network, Word2Vec has some hyperparameters. Let's play with some of these. 

# Word2Vec hyperparameters

❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the embedding size is the one you chose.

In [14]:
word2vec_2 = Word2Vec(sentences=X_train, vector_size=130)
wv2 = word2vec_2.wv

❓ **Question** ❓ Use the **`Word2Vec.wv.key_to_index`** attribute to display the size of the learned vocabulary. Compare it to the number of different words in `X_train`.

In [15]:
wv2.key_to_index

{'the': 0,
 'and': 1,
 'a': 2,
 'of': 3,
 'to': 4,
 'is': 5,
 'br': 6,
 'in': 7,
 'it': 8,
 'i': 9,
 'this': 10,
 'that': 11,
 'was': 12,
 'as': 13,
 'for': 14,
 'with': 15,
 'movie': 16,
 'but': 17,
 'film': 18,
 'on': 19,
 'not': 20,
 'you': 21,
 'his': 22,
 'are': 23,
 'have': 24,
 'be': 25,
 'he': 26,
 'one': 27,
 'all': 28,
 'at': 29,
 'by': 30,
 'they': 31,
 'an': 32,
 'so': 33,
 'like': 34,
 'from': 35,
 'who': 36,
 'her': 37,
 'or': 38,
 'just': 39,
 'out': 40,
 'if': 41,
 "it's": 42,
 'about': 43,
 'has': 44,
 'some': 45,
 'there': 46,
 'what': 47,
 'good': 48,
 'more': 49,
 'when': 50,
 'very': 51,
 'up': 52,
 'my': 53,
 'time': 54,
 'she': 55,
 'even': 56,
 'no': 57,
 'would': 58,
 'really': 59,
 'which': 60,
 'only': 61,
 'see': 62,
 'their': 63,
 'had': 64,
 'story': 65,
 'can': 66,
 'me': 67,
 'were': 68,
 'well': 69,
 'than': 70,
 'we': 71,
 'much': 72,
 'bad': 73,
 'get': 74,
 'been': 75,
 'do': 76,
 'people': 77,
 'will': 78,
 'into': 79,
 'also': 80,
 'first': 81,
 'b

There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been trained on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec:  `min_count`. 

`min_count` is a integer that tells you how many occurrences a given word should have to be learned in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to learn a representation of words that are sufficiently present in the corpus to have a robust embedded representation.

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vecs that you have trained (you can choose any `vector_size` you want).

In [16]:
word2vec_3 = Word2Vec(sentences=X_train, vector_size=130, min_count=3)
wv3 = word2vec_3.wv
word2vec_4 = Word2Vec(sentences=X_train, vector_size=130, min_count=10)
wv4 = word2vec_4.wv

print(len(wv3),len(wv4))

20607 8817


Remember that Word2Vec has an internal neural network that is optimized based on some predictions. These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the Word2Vec with different `window` sizes.

❓ **Question** ❓ Train a new `word2vec_5` model with a `window` different than previously (default is 5).

In [17]:
word2vec_5 = Word2Vec(sentences=X_train, vector_size=130, min_count=7, window=7)
wv5 = word2vec_5.wv

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the ones that you should start playing with to get a better performance for your model.

But you can also look at other arguments in the [**📚 Documentation - gensim.models.word2vec.Text8Corpus**](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)

# Convert our train and test set to RNN-ready datasets

Remember that `Word2Vec` is the first step to the overall process of feeding such a representation into a RNN, as shown here:

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be fed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (make sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- For the <font color=orange>test</font> set, this is understandable: <font color=orange>some words were not</font> in the <font color=blue>train</font> set and thus, their <font color=orange>embedded representation is unknown</font>
- for the <font color=blue>train set</font>, due to `min_count` hyperparameter, not all the words have a vector representation.

In any case, just skip the missing words here.

In [18]:
'ame' in wv

False

In [19]:
def embed_sentence(word2vec, sentence):
    return np.array([word2vec[i] for i in sentence if i in word2vec]) 

In [20]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']


    
### Checks
embedded_sentence = embed_sentence(wv, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(wv, example_missing_words)  
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

❓ **Question** ❓ Write a function that, given a list of sentences (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

💡 _Hint_: Use the previous function `embed_sentence`

In [ ]:
def embedding(word2vec, sentences):
    return [embed_sentence(word2vec, i) for i in sentences]
    
X_train_emb = embedding(wv, X_train)
X_test_emb = embedding(wv, X_test)

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad your sequences so you have tensors which can be divided into batches (of `batch_size`) during the optimization. Store the padded values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [ ]:

def plot_hist(X):
    len_ = [len(_) for _ in X]
    plt.hist(len_)
    plt.title('Histogram of the number of sentences that have a given number of words')
    plt.show()
    
plot_hist(X_train_emb)

In [ ]:

X_train_pad = pad_sequences(X_train_emb, dtype='float32', padding='post', value=0, maxlen=400)
X_test_pad = pad_sequences(X_test_emb, dtype='float32', padding='post', value=0, maxlen=400)

assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)



🏁 Congratulations, you are now able to use `Word2Vec` to embed your words :)

💾 Don't forget to git add/commit/push your notebook...

🚀 ... and move on to the next challenge!


In [ ]:
!git Understand-embeddings-with-word2vec.ipynb

!git commit -m 'Challenge validation nNLP beginner, word2vec'

!git push origin master